# BERT를 활용한 Dense Passage Retrieval 실습

### Requirements

In [1]:
!pip install datasets
!pip install transformers

## 데이터셋 로딩


KorQuAD train 데이터셋을 학습 데이터로 활용

In [2]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/92f88eedc7d67b3f38389e8682eabe68caa450442cc4f7370a27873dbc045fe4)


## 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, hugginface에서 제공하는 "bert-base-multilingual-cased" tokenizer를 활용

In [3]:
from transformers import AutoTokenizer
import numpy as np

model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [4]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [5]:
tokenized_input = tokenizer(dataset['train'][0]['context'], padding="max_length", truncation=True)
tokenizer.decode(tokenized_input['input_ids'])

'[CLS] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

## Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [6]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
np.random.seed(2021)
random.seed(2021)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [7]:
# To do: Train with full dataset
training_dataset = dataset['train'].select(range(1000))

In [8]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')


In [9]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러오기

In [10]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output


In [11]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias',

Train function 정의 후, 두개의 encoder fine-tuning 하기 (In-batch negative 활용) 


In [15]:
def train(args, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  optimizer = AdamW([
                {'params': p_model.parameters()},
                {'params': q_model.parameters()}
            ], lr=args.learning_rate, weight_decay=args.weight_decay)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)

      p_inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2]
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}
      
      p_outputs = p_model(**p_inputs)  # (batch_size, emb_dim)
      q_outputs = q_model(**q_inputs)  # (batch_size, emb_dim)


      # Calculate similarity score & loss
      sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)

      # target: position of positive samples = diagonal element 
      targets = torch.arange(0, args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        targets = targets.to('cuda')

      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)
      print(loss)
      print(len(batch[4]), batch[4].shape)
      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()


    
  return p_model, q_model

In [13]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01
)


In [ ]:
p_encoder, q_encoder = train(args, train_dataset, p_encoder, q_encoder)

Iteration:   0%|          | 0/250 [00:00<?, ?it/s]

tensor(0.2175, device='cuda:0', grad_fn=<NllLossBackward>)
4 torch.Size([4, 512])



Iteration:   0%|          | 1/250 [00:00<03:08,  1.32it/s]

tensor(0.1957, device='cuda:0', grad_fn=<NllLossBackward>)
4 torch.Size([4, 512])



Iteration:   1%|          | 2/250 [00:01<03:10,  1.30it/s]

...

tensor(0.0506, device='cuda:0', grad_fn=<NllLossBackward>)
4 torch.Size([4, 512])



Iteration:  28%|██▊       | 71/250 [00:56<02:21,  1.27it/s]

tensor(0.0873, device='cuda:0', grad_fn=<NllLossBackward>)
4 torch.Size([4, 512])



Iteration:  29%|██▉       | 72/250 [00:56<02:20,  1.26it/s]

tensor(0.3659, device='cuda:0', grad_fn=<NllLossBackward>)
4 torch.Size([4, 512])


## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [ ]:

valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth, '\n\n')

# valid_corpus

GP는 무엇의 약자인가?
GP-슈트레케는 1984년 공사를 통해 새로 만들어진 트랙이다. 1984년 이전에는 쥐트슐라이페라는 이름의 트랙이 있었다. 기존의 피트공간과 쥐트슐라이페의 일부분을 사용해 GP-슈트레케가 탄생한다. GP는 그랑프리의 약자로, 처음부터 포뮬러 원 개최를 목표로 만들어졌다. 과거에 비해 크기는 많이 줄어들었고 안전성을 높이는 방향으로 건설되었다. 하지만 이 때문에, 당시에는 GP-슈트레케는 뉘르부르크링의 진가를 발휘할 수 없다는 의견도 있었다. 하지만 그 후 포뮬러 원, 룩셈부르크 그랑프리, 유럽 그랑프리를 성공적으로 개최해나가면서, 지금은 노르트슐라이페에 못지 않는 국제적으로 유명한 서킷이 되었다. 현재 쥐트슐라이페의 일부분은 남아있으나 트랙으로 사용되지 않고 있다. 




앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

In [ ]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [ ]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([11, 768]) torch.Size([1, 768])


생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

In [ ]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

torch.Size([1, 11])
tensor([[-10.1625,  -7.7045,  -8.0828,  -9.6541, -10.6419, -10.1482,  -6.6765,
          -7.7555,  -8.5440,  -7.7525,  -2.8934]])
tensor([10,  6,  1,  9,  7,  2,  8,  3,  5,  0,  4])


Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [ ]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 GP는 무엇의 약자인가? 

[Ground truth passage]
GP-슈트레케는 1984년 공사를 통해 새로 만들어진 트랙이다. 1984년 이전에는 쥐트슐라이페라는 이름의 트랙이 있었다. 기존의 피트공간과 쥐트슐라이페의 일부분을 사용해 GP-슈트레케가 탄생한다. GP는 그랑프리의 약자로, 처음부터 포뮬러 원 개최를 목표로 만들어졌다. 과거에 비해 크기는 많이 줄어들었고 안전성을 높이는 방향으로 건설되었다. 하지만 이 때문에, 당시에는 GP-슈트레케는 뉘르부르크링의 진가를 발휘할 수 없다는 의견도 있었다. 하지만 그 후 포뮬러 원, 룩셈부르크 그랑프리, 유럽 그랑프리를 성공적으로 개최해나가면서, 지금은 노르트슐라이페에 못지 않는 국제적으로 유명한 서킷이 되었다. 현재 쥐트슐라이페의 일부분은 남아있으나 트랙으로 사용되지 않고 있다. 

Top-1 passage with score -2.8934
GP-슈트레케는 1984년 공사를 통해 새로 만들어진 트랙이다. 1984년 이전에는 쥐트슐라이페라는 이름의 트랙이 있었다. 기존의 피트공간과 쥐트슐라이페의 일부분을 사용해 GP-슈트레케가 탄생한다. GP는 그랑프리의 약자로, 처음부터 포뮬러 원 개최를 목표로 만들어졌다. 과거에 비해 크기는 많이 줄어들었고 안전성을 높이는 방향으로 건설되었다. 하지만 이 때문에, 당시에는 GP-슈트레케는 뉘르부르크링의 진가를 발휘할 수 없다는 의견도 있었다. 하지만 그 후 포뮬러 원, 룩셈부르크 그랑프리, 유럽 그랑프리를 성공적으로 개최해나가면서, 지금은 노르트슐라이페에 못지 않는 국제적으로 유명한 서킷이 되었다. 현재 쥐트슐라이페의 일부분은 남아있으나 트랙으로 사용되지 않고 있다.
Top-2 passage with score -6.6765
2017년 9월 2일 방탄소년단은 서울올림픽주경기장에서 열린 서태지 데뷔 25주년 기념 공연 ‘롯데카드 무브ː사운드트랙 vol.2 서태지 25’에 스페셜 게스트로 참여해 무대에 올랐다. 서태지의 공식적인 초대를 받아 스페

## Wikipedia documents에 대해 Dense Passage Retrieval 실습하기

In [ ]:
# First load wikipedia dump
import json

# TODO: Write your own path and load json file
dump_path = ...
with open(dump_path, 'r') as f:
    wiki = json.load(f)

In [ ]:
len(wiki), type(wiki)

In [ ]:
wiki['0']

In [ ]:
# wiki_embs = embedded dense vectors of documents
wiki_embs = ...


In [ ]:
# Answer: (60613, 768) (num_doc, emb_dim)
wiki_embs.shape

In [ ]:
# TODO: embed query to dense vector
query = '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'
q_emb = ...

In [ ]:
result = torch.matmul(q_emb, torch.transpose(wiki_embs, 0, 1))
result.shape

In [ ]:
k = 5
rank = torch.argsort(result, dim=1, descending=True).squeeze()

In [ ]:
k = 5
print("[Search query]\n", query, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, result.squeeze()[rank[i]]))
  print(corpus[rank[i]])